In [1]:
KotlinVersion.CURRENT             // 코틀린 현재 버전 

1.8.0

In [2]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.6.4")

In [3]:
:classpath

Current classpath (16 paths):
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/lib-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/api-0.11.0-170.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-script-runtime-1.8.0-dev-3517.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-reflect-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlin-stdlib-common-1.7.10.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/annotations-13.0.jar
/Users/a06411/opt/anaconda3/envs/kotlin/lib/python3.10/site-packages/run_kotlin_kernel/jars/kotlinx-serialization-json-jvm-1.3.3.jar
/Users/a06

In [4]:
import kotlinx.coroutines.*

# 1. 코루틴 알아보기

## 1-1 코루틴 기본 알아보기

## 1-1-1 일시중단함수 

- 함수 앞에 suspend 키워드를 사용해서 코루틴에서 처리하는 일시 중단함수를 작성할 수 있다. 
- 보통 일시중단함수는 호출한 순서대로 순차적으로 실행한다.  일시중단할 때 스레드를 블러킹하지 않는다 

In [5]:
suspend fun getName() : String {
    delay(100)
    return "코루틴"
}

suspend fun getLastName() : String {
    delay(100)
    return "코틀린"
}

### 코루틴이 실행되는 영역을 위해 runBlocking으로 영역을 설정

- 코루틴 영역이 설정해야 내부에서 일시중단함수 등이 실행된다. 

In [6]:
fun main() = runBlocking {
    val name = getName()
    val lastName = getLastName()
    println("Hello, $name $lastName")
}

In [7]:
main()

Hello, 코루틴 코틀린


### 코루틴 빌더 : 동시에 일시중단함수를 호출하기

In [8]:
fun main() = runBlocking {
    val name = launch {getName() }
    val lastName = launch {getLastName()}
    println("Hello, $name $lastName")
}

In [9]:
main()

Hello, StandaloneCoroutine{Active}@1c5c84a9 StandaloneCoroutine{Active}@656406e4


### 코루틴 빌더 : 동시에 실행하고 반환값 처리 

In [10]:
fun main() = runBlocking {
    val name = async {getName() }
    val lastName = async {getLastName()}
    println("Hello, ${name.await()} ${lastName.await()}")
}

In [11]:
main()

Hello, 코루틴 코틀린


## 1-1-2 코루틴 빌더로 코루틴 실행 함수 정의 

In [12]:
import kotlinx.coroutines.*                               // 코루틴 패키지

suspend fun createCoroutines(amount:Int)= runBlocking { 
    var jobs = ArrayList<Job>()                           // 배열의 작업 결과를 저장
    for (i in 1..amount) {
        jobs += launch {                                  // 코루틴 처리 후 결과 저장 
            print("...")
            delay(1000)
        }
    }
    
    jobs.forEach {
        it.join()                                         // 코루틴 실행할 때까지 기다린다. 
    }
    println()
    println(jobs)                                         // 코루틴 처리 결과 
}


In [13]:
fun main() = runBlocking {
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    print("start")
    createCoroutines(10)                                  // 코루틴 일시중단함수 실행 
    println("end")
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
}

In [14]:
main()

코루틴 처리 : [BlockingCoroutine{Active}@51e68ee, BlockingEventLoop@5fc4e721]
start..............................
[StandaloneCoroutine{Completed}@42a2e922, StandaloneCoroutine{Completed}@5c2ffffb, StandaloneCoroutine{Completed}@73a98bd2, StandaloneCoroutine{Completed}@570e8451, StandaloneCoroutine{Completed}@2308c6b, StandaloneCoroutine{Completed}@21c37bf1, StandaloneCoroutine{Completed}@1e8b9300, StandaloneCoroutine{Completed}@3d04a08c, StandaloneCoroutine{Completed}@3bd12093, StandaloneCoroutine{Completed}@1f7d8eb]
end
코루틴 처리 : [BlockingCoroutine{Active}@51e68ee, BlockingEventLoop@5fc4e721]


## 1-1-3  일반함수와 일시중단함수를 코루틴에서 처리

In [15]:
fun task1() = println("일반함수 task 1 : " 
                     + Thread.currentThread().name)       // 첫번째 함수 

fun task2() = println("일반함수 task 2 : "                  // 두번째 함수 
                     + Thread.currentThread().name)

In [16]:
suspend fun task3() =  println("일수중단함수 task 3 : "      // 일시중단함수 정의
                     + Thread.currentThread().name)

val task4 = suspend {println("일시중단함수 task 4 : "        // 람다표현식도 일시중단함수 정의 가능 
                     + Thread.currentThread().name)}

In [17]:
val rb = runBlocking { 
    println("코루틴 처리 : " + coroutineContext)             // 코루틴 환경구성 
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task1()                                           // 일반함수 호출 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task2()                                           // 일반함수 호출 
        delay(300)
    }
    launch {                                              // 코루틴 빌더로 함수처리
        delay(300)
        task3()                                           // 일시중단함수 
    }
    launch {                                              // 코루틴 빌더로 함수처리
        task4()                                           // 일시중단함수
        delay(300)
    }
}                                                         // 메인 스레드 사용

코루틴 처리 : [BlockingCoroutine{Active}@c70acc6, BlockingEventLoop@648c98e6]
일반함수 task 2 : Thread-65
일시중단함수 task 4 : Thread-65
일반함수 task 1 : Thread-65
일수중단함수 task 3 : Thread-65


## 1-2 디스패처 알아보기¶

- Dispatcher는 CoroutineContext를 상속받아 어떤 쓰레드를 이용해서 동작할 것인지 정의해둔 것 입니다.

- None : 실행된 코루틴 스코프에게 상속받는다.
- Unconfined : 호출 쓰레드에서 실행하지만 멈췄다가 재개할 경우 재개한 쓰레드에서 실행한다.
             -> 특정 쓰레드에 국한되지 않는 경우 적합

- Main : 메인 쓰레드에서 실행한다.
         -> 안드로이드에서 UI를 바꿀 때 적합, Job이 무거우면 부적합

- IO : 네트워크, 디스크 사용 할때 사용합니다. 파일 읽고, 쓰고, 소켓을 읽고, 쓰고 작업을 멈추는것에 최적화되어 있습니다.
- Default : Thread Pool에 할당한다. (GlobalScope.launch { }도 마찬가지)

## 1-2-1 디스패처 상태 확인하기 

In [18]:
fun main() { 
    runBlocking {
        launch {
            println(" 미지정 일때  : " + coroutineContext)             // 코루틴 환경구성 
            println(" launch(미지정) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Unconfined) {
            println("Unconfined : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Unconfined) : ${Thread.currentThread().name}")
        }

       //launch(Dispatchers.Main) {   //안드로이드 용
       //    println("launch(Dispatchers.Main) : ${Thread.currentThread().name}")
       //}

        launch(Dispatchers.IO) {
            println(" IO : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(IO) : ${Thread.currentThread().name}")
        }

        launch(Dispatchers.Default) {
            println("Default : " + coroutineContext)             // 코루틴 환경구성 
            println("launch(Default) : ${Thread.currentThread().name}")
        }
    }
}

main()

Unconfined : [StandaloneCoroutine{Active}@53bc7769, Dispatchers.Unconfined]
launch(Unconfined) : Thread-69
 IO : [StandaloneCoroutine{Active}@59d9ea19, Dispatchers.IO]
launch(IO) : DefaultDispatcher-worker-1
Default : [StandaloneCoroutine{Active}@7f96a50b, Dispatchers.Default]
 미지정 일때  : [StandaloneCoroutine{Active}@6d290f28, BlockingEventLoop@6ae0f1ce]
launch(Default) : DefaultDispatcher-worker-3
 launch(미지정) : Thread-69


## 1-2-2 코루틴 컨텍스트 조회하기 

In [19]:
fun main() = runBlocking {                                     // 런블러킹 컨텍스트 확인 
    println("runBlocking : ${Thread.currentThread().name}")    
    println("컨텍스트 : " + coroutineContext)                                      
    println("상태 : " +isActive)                                // 잡상태 확인
    
    val g = GlobalScope.launch {                               // 전역스코프 컨텍스트 확인 
        delay(300)
        println("전역스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                            // 잡상태 확인 
    }
    println("전역스코프 반환 : " + g)
    val c =CoroutineScope(Dispatchers.Default).launch {        // 코루틴스코프 컨텍스트 확인
        delay(500)
        println("코루틴 스코프 : ${Thread.currentThread().name}")
        println("컨텍스트 : " +coroutineContext)
        println("상태 : " +isActive)                             // 잡상태 확인    
    }
    println("코르틴 스코프 반환 : " + c)
    delay(1000)
}

main()

runBlocking : Thread-76
컨텍스트 : [BlockingCoroutine{Active}@6466089a, BlockingEventLoop@4a93974c]
상태 : true
전역스코프 반환 : StandaloneCoroutine{Active}@19498c3a
코르틴 스코프 반환 : StandaloneCoroutine{Active}@50ae342c
전역스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@19498c3a, Dispatchers.Default]
상태 : true
코루틴 스코프 : DefaultDispatcher-worker-3
컨텍스트 : [StandaloneCoroutine{Active}@50ae342c, Dispatchers.Default]
상태 : true


## 1-2-3  스레드와 코루틴 실행 비교하기 

In [20]:
fun main() {
    Thread(Runnable {                                   // 3개의 러너블 객체를 만들어서 스레드 처리               
        for (i in 0..2) {
            println("쓰레드 처리  : $i "                   // 3개의 쓰레드를 처리 
                    + Thread.currentThread().name)
            Thread.sleep(500L)                          // 스레드 중단 : 현재 스레드 일시중단 
        }
    }).start()                                          // start 메소드로 스레드 시작

    GlobalScope.launch {                                // 코루틴 전역스코프로 처리 
        repeat(3) {                                     // 3개의 일지중단함수 만들고 코루틴 생성 
            println("코루틴처리   : $it " 
                    + Thread.currentThread().name)      // 코루틴 내의 dispatchre로 처리 
            delay(500L)                                 // 일시 중단 다른 작업 수행 
        }
    }
    
    Thread.sleep(3000L)                                 // 어플리케이션 중단 
}

main()

쓰레드 처리  : 0 Thread-81
코루틴처리   : 0 DefaultDispatcher-worker-3
쓰레드 처리  : 1 Thread-81
코루틴처리   : 1 DefaultDispatcher-worker-3
코루틴처리   : 2 DefaultDispatcher-worker-3
쓰레드 처리  : 2 Thread-81
